In [ ]:
#default_exp utils

In [ ]:
#hide
from nbdev.showdoc import *

In [8]:
#hide
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [1]:
#export 
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors


try:
    from sparse_dot_topn import awesome_cossim_topn
except ImportError as e:
    warn(f"{e}")
    warn("sparse_dot_topn module not installed, will use naive dot product for vector query. This may lead to memory overload")
    

In [12]:
#export



def pad_to_shape(array, shape):
    '''fill array with trilling zeros to match shape'''
    arr = np.zeros(shape)    
    arr[tuple(slice(0,i) for i in array.shape)] = array
    return arr



def cos_sim_query(query_vector, query_space, n_neighbors=50, lower_bound=0.0):    
    '''make cos similarity query of query_vector on query_space'''
    try:
        sim_matrix = (awesome_cossim_topn(normalize(query_vector), normalize(query_space).T,
                                         ntop=n_neighbors, lower_bound=lower_bound))     
        
        sim_matrix = sim_matrix.tocoo()
    
        idx = []
        sim = []
        arr_sizes = []
        for row in range(sim_matrix.shape[0]):
            s = sim_matrix.data[sim_matrix.row == row]
            i = sim_matrix.col[sim_matrix.row == row]
            sim.append(s)
            idx.append(i)
            arr_sizes.append(len(s))

        max_size = max(arr_sizes)
        idx = np.array([pad_to_shape(i, max_size) for i in idx]).astype(int)
        sim = np.array([pad_to_shape(s, max_size) for i in sim])
        if idx.shape[1] == 0:
            raise ValueError('No similarity greater than lower_bound found. Choose a lower threshold.')
        return  idx, sim
    
    except NameError:
        dist, idx = (
            NearestNeighbors(n_neighbors = n_neighbors, radius = 1 - lower_bound, metric = 'cosine', n_jobs = -1)
            .fit(normalize(query_space))
            .kneighbors(normalize(query_vector))
        )
        return idx, 1 - dist # <- cos_sim = 1 - cos_dist
        

def sample_multi_dim(arr, size, weights, replace = True, axis = 0):
    '''
    function that extends np.random.choice to work with arbitrary axis sampling
    len(weight) must be equal to n elements in axis
    '''
    assert len(weights) == arr.shape[axis]
    sampled_idxs = np.random.choice([*range(arr.shape[axis])], size = size, p = weights, replace = replace)
    return np.take(arr, sampled_idxs, axis=axis)
    

    
def ctqdm(iterable, verbose = False, notebook = False,**tqdm_kwargs):
    '''
    progress bar handler (custom tqdm)
    '''
    if not verbose:
        return iterable
    else:
        if notebook:
            return tqdm.notebook(iterable,**tqdm_kwargs)
        else:
            return tqdm.tqdm(iterable,**tqdm_kwargs)

# Export -

In [13]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_ensemble.ipynb.
Converted 02_core.random_variable.ipynb.
Converted 03_utils.ipynb.
